## 5-2　BERT を用いた文章の穴埋め

In [1]:
# !pip install transformers==4.18.0 fugashi===1.1.0 ipadic==1.0.0

In [2]:
import torch
import numpy as np
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [3]:
model_name = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [5]:
input_ids = tokenizer.encode(text)
print(input_ids)
print(tokenizer.convert_ids_to_tokens(input_ids))

[2, 3246, 9, 4, 118, 3488, 8, 3]
['[CLS]', '今日', 'は', '[MASK]', 'へ', '行く', '。', '[SEP]']


In [6]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

with torch.no_grad():
  output = bert_mlm(input_ids=input_ids)
  scores = output.logits

In [7]:
mask_position = input_ids[0].tolist().index(4)
print(mask_position)

id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

text = text.replace('[MASK]', token_best)
print(text)

3
今日は東京へ行く。


In [8]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
  input_ids = tokenizer.encode(text, return_tensors='pt')
  input_ids = input_ids.cuda()
  with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
  scores = output.logits

  mask_position = input_ids[0].tolist().index(4)
  topk = scores[0, mask_position].topk(num_topk)
  ids_topk = topk.indices
  tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
  scores_topk = topk.values.cpu().numpy()

  text_topk = []
  for token in tokens_topk:
    token = token.replace('##', '')
    text_topk.append(text.replace('[MASK]', token, 1))

  return text_topk, scores_topk

In [9]:
text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [10]:
print(*_, sep='\n')

9.178565
9.145953
8.923241
8.838814
8.319214
8.180512
7.917591
7.8333464
7.826693
7.8070245


In [11]:
text_topk

['今日は東京へ行く。',
 '今日はハワイへ行く。',
 '今日は学校へ行く。',
 '今日はニューヨークへ行く。',
 '今日はどこへ行く。',
 '今日は空港へ行く。',
 '今日はアメリカへ行く。',
 '今日は病院へ行く。',
 '今日はそこへ行く。',
 '今日はロンドンへ行く。']

In [12]:
_

array([9.178565 , 9.145953 , 8.923241 , 8.838814 , 8.319214 , 8.180512 ,
       7.917591 , 7.8333464, 7.826693 , 7.8070245], dtype=float32)

In [13]:
def greedy_prediction(text, tokenizer, bert_mlm):
  for _ in range(text.count('[MASK]')):
    text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
  return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [14]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は社会社会的な地位'

In [15]:
def beam_search(text, tokenizer, bert_mlm, num_topk):
  num_mask = text.count('[MASK]')
  text_topk = [text]
  scores_topk = np.array([0])

  for _ in range(num_mask):
    text_candidates = []
    score_candidates = []
    for text_mask, score in zip(text_topk, scores_topk):
      text_topk_inner, scores_topk_inner = predict_mask_topk(text_mask, tokenizer, bert_mlm, num_topk)
      text_candidates.extend(text_topk_inner)
      score_candidates.append(score + scores_topk_inner)

    score_candidates = np.hstack(score_candidates)
    idx_list = score_candidates.argsort()[::-1][:num_topk]
    text_topk = [text_candidates[idx] for idx in idx_list]
    scores_topk = score_candidates[idx_list]

  return text_topk

In [16]:
text = '今日は[MASK][MASK]へ行く。'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [17]:
text_candidates = []

a = [1, 2, 3]
text_candidates.extend(a)

b = [4, 5, 6]
text_candidates.extend(b)

print(text_candidates)

[1, 2, 3, 4, 5, 6]


In [18]:
score_candidates = []

a = np.array([1])
b = np.array([2, 3, 4])
score_candidates.append(a + b)

c = np.array([1])
d = np.array([6, 7, 8])
score_candidates.append(c + d)

print(score_candidates)

score_candidates = np.hstack(score_candidates)
print(score_candidates)

[array([3, 4, 5]), array([7, 8, 9])]
[3 4 5 7 8 9]


In [19]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。
